In [1]:
import pickle
import numpy as np 
import matplotlib.pyplot as plt
from scipy.sparse import lil_matrix,triu,diags,csr_matrix
from sklearn.metrics import roc_auc_score, roc_curve
import gc
import itertools
import math
import ast
import pandas as pd

Faccio finta che sto lavorando su pat_emb_vero

In [2]:
embeddings_dataframe = pd.read_pickle('pat_emb_vero.pkl')

In [3]:
embeddings_dataframe

,Patent_ID,Embedding_Vector,APPLN_TITLE,APPLN_ABSTR,APPLN_YR,TYPE
0,0010986_abstract,"[-0.20840316, -0.004724785, 0.24631755, -0.562...",Title,The invention provides a process for the prepa...,1980,abstract
1,0015809_abstract,"[-0.21267296, 0.102392994, -0.12945805, -0.197...",Title,A system and process for decolorizing and puri...,1980,abstract
2,0019348_abstract,"[0.36953244, 0.2832081, 0.025119131, 0.0457142...",Title,One of three possible states of a device such ...,1980,abstract
3,0008493_abstract,"[-0.07755193, -0.4931815, -0.2956493, -0.06191...",Title,A delayed coking process in which a liquid hyd...,1980,abstract
4,0020167_abstract,"[-0.17143352, 0.122270994, -0.2879149, 0.04859...",Title,A bucket-wheel unloader mounted on board a shi...,1980,abstract
...,...,...,...,...,...,...
1838543,4258533_abstract,"[0.35442418, -0.092198774, 0.039284695, -0.363...",Title,A power supply system and a solar photovoltaic...,2023,abstract
1838544,4167498_abstract,"[-0.3309879, -0.45200855, -0.24081644, -0.5175...",Title,"In certain embodiments, a method performed by ...",2023,abstract
1838545,4197425_abstract,"[-0.1979754, -0.48123467, -0.041454077, -0.264...",Title,The present invention relates to computer-impl...,2023,abstract
1838546,4199683_abstract,"[-0.0006965021, 0.09556287, -0.15228365, -0.28...",Title,A foldable display device (1000) includes a di...,2023,abstract


In [4]:
#cose preliminare tipo eliminare colonne inutili
embeddings_dataframe = embeddings_dataframe.drop(columns=['APPLN_TITLE','APPLN_ABSTR','TYPE'])

Adesso rimuovo gli zeri prima del numero del patent_ID e tolgo anche _abstract, nel tuo caso toglie _claims

In [6]:
embeddings_dataframe['Patent_ID'] = embeddings_dataframe['Patent_ID'].str.extract(r'0*([0-9]+)')

In [7]:
embeddings_dataframe

,Patent_ID,Embedding_Vector,APPLN_YR
0,10986,"[-0.20840316, -0.004724785, 0.24631755, -0.562...",1980
1,15809,"[-0.21267296, 0.102392994, -0.12945805, -0.197...",1980
2,19348,"[0.36953244, 0.2832081, 0.025119131, 0.0457142...",1980
3,8493,"[-0.07755193, -0.4931815, -0.2956493, -0.06191...",1980
4,20167,"[-0.17143352, 0.122270994, -0.2879149, 0.04859...",1980
...,...,...,...
1838543,4258533,"[0.35442418, -0.092198774, 0.039284695, -0.363...",2023
1838544,4167498,"[-0.3309879, -0.45200855, -0.24081644, -0.5175...",2023
1838545,4197425,"[-0.1979754, -0.48123467, -0.041454077, -0.264...",2023
1838546,4199683,"[-0.0006965021, 0.09556287, -0.15228365, -0.28...",2023


Questo codice utilizza i file già estratti in data_year.csv che trovi nella cartella drive EP_IPC. 

In [8]:
cumulative_df = pd.DataFrame()

# Iterate over each year from 1980 to 2023
for year in range(1980, 2024):
    # Load the data for the current year
    data_ipc = pd.read_csv(f'data_{year}.csv')
    
    # Convert IPC column to actual lists
    data_ipc['ipc'] = data_ipc['ipc'].apply(ast.literal_eval)
    
    # Rename columns for merging
    data_ipc = data_ipc.rename(columns={'APPLN_ID_CLEAN': 'Patent_ID'})
    
    # Ensure both Patent_ID columns have the same data type
    data_ipc['Patent_ID'] = data_ipc['Patent_ID'].astype(str)  # Convert to string if embeddings_dataframe has strings
    embeddings_dataframe['Patent_ID'] = embeddings_dataframe['Patent_ID'].astype(str)  # Convert to string if needed

    # Merge the dataframes
    merged_df = pd.merge(
        embeddings_dataframe[embeddings_dataframe['APPLN_YR'] == year], 
        data_ipc[['Patent_ID', 'ipc']], 
        on='Patent_ID', 
        how='left'
    )
    
    # Append the result to the cumulative DataFrame
    cumulative_df = pd.concat([cumulative_df, merged_df], ignore_index=True)
    cumulative_df = cumulative_df.drop_duplicates(subset='Patent_ID')

In [11]:
cumulative_df

,Patent_ID,Embedding_Vector,APPLN_YR,ipc
0,10986,"[-0.20840316, -0.004724785, 0.24631755, -0.562...",1980,[C08F2/24]
2,15809,"[-0.21267296, 0.102392994, -0.12945805, -0.197...",1980,[C01B25/235]
4,19348,"[0.36953244, 0.2832081, 0.025119131, 0.0457142...",1980,"[H03K17/18, G10H7/00, G10H1/057, G10H1/18]"
6,8493,"[-0.07755193, -0.4931815, -0.2956493, -0.06191...",1980,[C10B55/00]
8,20167,"[-0.17143352, 0.122270994, -0.2879149, 0.04859...",1980,"[B63B27/22, B65G65/20]"
...,...,...,...,...
3678504,4258533,"[0.35442418, -0.092198774, 0.039284695, -0.363...",2023,[H02M7/5387]
3678506,4167498,"[-0.3309879, -0.45200855, -0.24081644, -0.5175...",2023,"[H04B7/06, H04B7/0456]"
3678508,4197425,"[-0.1979754, -0.48123467, -0.041454077, -0.264...",2023,"[A61B3/00, A61B3/028, A61B3/02, A61B3/032, A61..."
3678510,4199683,"[-0.0006965021, 0.09556287, -0.15228365, -0.28...",2023,"[H10K59/126, H10K77/10, H10K102/00]"


Tolgo i due punti

In [13]:
cumulative_df['ipc']=cumulative_df['ipc'].replace(':','/')


/tmp/ipykernel_2319760/3789996499.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cumulative_df['ipc']=cumulative_df['ipc'].replace(':','/')


Controllo che ogni elemento della colonna ipc sia una lista

In [16]:
type(cumulative_df['ipc'][0])

list

Controllo che ogni lista non abbia doppioni al suo interno 

In [17]:
cumulative_df['ipc'] = cumulative_df['ipc'].apply(lambda x: list(set(x)))

/tmp/ipykernel_2319760/2777029056.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cumulative_df['ipc'] = cumulative_df['ipc'].apply(lambda x: list(set(x)))


In [18]:
cumulative_df

,Patent_ID,Embedding_Vector,APPLN_YR,ipc
0,10986,"[-0.20840316, -0.004724785, 0.24631755, -0.562...",1980,[C08F2/24]
2,15809,"[-0.21267296, 0.102392994, -0.12945805, -0.197...",1980,[C01B25/235]
4,19348,"[0.36953244, 0.2832081, 0.025119131, 0.0457142...",1980,"[G10H1/057, G10H7/00, G10H1/18, H03K17/18]"
6,8493,"[-0.07755193, -0.4931815, -0.2956493, -0.06191...",1980,[C10B55/00]
8,20167,"[-0.17143352, 0.122270994, -0.2879149, 0.04859...",1980,"[B65G65/20, B63B27/22]"
...,...,...,...,...
3678504,4258533,"[0.35442418, -0.092198774, 0.039284695, -0.363...",2023,[H02M7/5387]
3678506,4167498,"[-0.3309879, -0.45200855, -0.24081644, -0.5175...",2023,"[H04B7/0456, H04B7/06]"
3678508,4197425,"[-0.1979754, -0.48123467, -0.041454077, -0.264...",2023,"[A61B3/02, A61B3/028, A61B3/032, A61B3/00, A61..."
3678510,4199683,"[-0.0006965021, 0.09556287, -0.15228365, -0.28...",2023,"[H10K102/00, H10K77/10, H10K59/126]"
